# Modules, documentation and testing


<img src="https://imgs.xkcd.com/comics/random_number.png" style="width: 800px;"/>


 **Why document your code?**

Of paramount importance to produce good code **understandable by others and the author**, and hence maintainable in the long run.

If you want to learn more on how to write comments, I recommend reading this blog post: http://antirez.com/news/124

# Modules

### What is a Python module?
A module is a file consisting of Python code. A module can define functions, classes and variables. A module can also include runnable code.

### What is it good for?

Use modules to organize your program logically
  * Split the code into several files for easier maintenance.
  * Group related code into a module.
  * Share common code between scripts.
  * Publish modules on the web for other people to use.

## Using modules

Python comes with already with many modules that you can use.
For example, let's import the module called `sys` and access its `argv` variable:

```python
import sys
x = float(sys.argv[1])
```

Import module member `argv` into current namespace:

```python
from sys import argv
x = float(argv[1])
```

Import everything from `sys` (not recommended):

```python
from sys import *
x = float(argv[1])

flags = ''
# Ooops, flags was also imported from sys, this new flags
# name overwrites sys.flags!
```

Import `argv` under an alias:

```python
from sys import argv as a
x = float(a[1])
```

## Making your own Python modules


 * Put classes and functions that should be part of the module in a Python file (e.g. `my_module.py`)

Examples:

```python
import my_module
# or
import my_module as mm   # M is a short form
# or
from my_module import *  # import all variables/classes/functions 
# or
from my_module import my_function
```

## Test block in a module


Module files can have a test/demo section at the end:
    

```python
if __name__ == '__main__':
    infile = sys.argv[1]; outfile = sys.argv[2]
    for i in sys.argv[3:]:
        create(infile, outfile, i)
```        

* The block is executed *only if* the module file is run as a program (not if imported by another script)
* The tests at the end of a module often serve as good examples on the usage of the module

## Packages

 * A set of modules can be collected in a *package*
 * A package is organized as module files in a directory tree
 * Each subdirectory must have a `__init__.py` file  (can be empty)
 * More infos: [Section 6 in the Python Tutorial](https://docs.python.org/3/tutorial/modules.html)  

## Packages

Example directory tree of a Python package:

```bash
pde_solver/
   __init__.py
   numerics/
       __init__.py
       pde/
           __init__.py
           grids.py     # contains fdm_grids object
```

Can import modules in the tree like this:
    

```python
from pde_solver.numerics.pde.grids import fdm_grids

grid = fdm_grids()
grid.domain(xmin=0, xmax=1, ymin=0, ymax=1)
...
```

## How does Python find your packages/modules?


When importing a package or module, Python tries to find it in multiples places, including:

* `/usr/lib/python3.5/site-packages`       <-- This depends on your OS and Python version
* paths defined by the environment variable `PYTHONPATH`
* your current working directory

## Option 1: Adding your module path to PYTHONPATH

Let's assume that your module is stored in the path `/home/simon/mymod/`. In your current (!) Bash session, you can add this path to the Python search with:


```bash
> export PYTHONPATH=/home/simon/mymod:$PYTHONPATH
```

# Option 2: Create a setup.py file

Create a `setup.py` file in your package/module directory.

```python
from distutils.core import setup

setup(
    name='PDE Solver',
    version='1.0',
    packages=['pde_solver',]
)
```



and install with 
```bash
pip3 install . --user  # For single-user installation
pip3 install .         # For system wide installation
```

The directory tree of the final Python project is then:

```bash
PDESolver/
    setup.py          # setup/installation script
    requirements.txt  # list package dependencies here
    README.txt
    pde_solver/       # The actual Python package/modules
       __init__.py
       numerics/
           __init__.py
           pde/
               __init__.py
               grids.py     # contains fdm_grids object
```

# pip can install external packages

Similar, to the `conda` command, `pip` can be used to install additional Python packages:
```bash
pip3 install scipy --user  # Install from the Python package index PyPI
pip3 install git+https://bitbucket.org/dolfin-adjoint/pyadjoint.git  # Install from a git repository
``` 

## Docstrings - Document your code!

Python treats a string in the first line of a module/function/class definition as a special **documentation string**.:

In [3]:
"""
A collection of mathematical functions.
"""

from math import sin

def minsin(x):
    """ Calculates the sin of a number and returns the result
    
    A more detailed description goes here.
    """
    return sin(x)
    

**Docstring guideline**: The first line should always be a short, concise summary of the functions’s purpose. A more detailed description can follow below seperated by a newline.

See http://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html for a complete docstring example.
    

* Some code editors will present this docstring for you on request. For example in `IPython (notebook)`:

* You can also explicitely access the doc string:

In [4]:
minsin?

# Testing


## Why should we test?

* To check correctness of software.
* To ensure that future changes do not break functionality.
* To check if the software runs succesfully in a different environment (newer Python version, upgraded libraries, different operating system)

## A few options in Python

* [Unittest](https://docs.python.org/3/library/unittest.html)

* [Doctest](https://docs.python.org/3/library/doctest.html)
* [Py.test](http://pytest.org/) (will be used here)

## How to use py.test

Say you have a function `absolute_value` in a file that needs testing:
```python
# script.py 
def absolute_value(x):
    if x < 0:
        return x
    else:
        return -x
```        

Create a associated test file `test_script.py`:

In [7]:
# test_script.py
from script import absolute_value    # Import the function 

def test_funcs():                    # py.test will automatically run all functions starting with test_
    assert absolute_value(-3) == 3   # Add some tests here...
    assert absolute_value(5)  == 5   # If one of the assert's evaluate to False, the test will fail
    assert absolute_value(0)  == 0    

In [8]:
!py.test test_script.py -v

============================= test session starts ==============================
platform linux -- Python 3.5.2, pytest-3.0.4, py-1.4.31, pluggy-0.4.0 -- /home/sf1409/miniconda3/bin/python
cachedir: .cache
rootdir: /home/sf1409/Documents/inf3331/UiO-INF3331.github.io/lectures/04-python-summary2, inifile: 
collected 1 items 

test_script.py::test_func FAILED

=================================== FAILURES ===================================
__________________________________ test_func ___________________________________

    def test_func():
>       assert absolute_value(-3) == 3
E       assert -3 == 3
E        +  where -3 = absolute_value(-3)

test_script.py:4: AssertionError
=========================== 1 failed in 0.03 seconds ===========================


# Using py.test
Let's fix our implementation...

In [9]:
!cat script.py

def absolute_value(x):
    if x < 0:
        return -x
    else:
        return x


and run the tests again

In [14]:
!py.test test_script.py -v

============================= test session starts ==============================
platform linux2 -- Python 2.7.15rc1, pytest-3.3.2, py-1.5.2, pluggy-0.6.0 -- /usr/bin/python2
cachedir: .cache
rootdir: /media/simon/Data/simon/Documents/inf3331/UiO-INF3331.github.io/lectures/04-python-summary2, inifile:
plugins: xdist-1.22.1, forked-0.2
collected 1 item                                                               

test_script.py::test_func PASSED                                         [100%]

=========================== 1 passed in 0.00 seconds ===========================


## Good testing practices

* Add new test while you develop new features.
* Make each test an unique stand alone example.
* Making tests resource undemanding.
* Run test suite before each commit-push.
* Make test function names descriptive.
* Quick way to learn other peoples code is through test suits.